In [1]:
import numpy as np

In [2]:
#sigma_a = 0.5 # process noise
sigma_m = 0.9 # measurement noise
x = np.array([72.5, 28.5, 2, 1]).T
z = np.array([75.0, 30]).T
omega = 35
alpha = 0.8
u = np.array([alpha])
rad = 0.5

P = np.array([[1, 0.0, 0.0, 0.0],
              [0.0, 1, 0.0, 0.0],
              [0.0, 0.0, 1, 0.0],
              [0.0, 0.0, 0.0, 1]])

Q = np.array([[alpha*rad, 0],
             [0, alpha*rad]])
R = np.array([[sigma_m, 0],
             [0, sigma_m]])


In [3]:
#number of dimensions since there are 4 elements in x n sigma = 8
dim = x.shape[0]
n_sigma = 1 + 2 * dim


# Now there are weights first is Wo which applied to the mean and other is Wi which is applied to all sigma points
# Wo = kappa/(kappa + dim)
# Wi = 0.5/(kappa + dim)
kappa = 3 - dim
Wo = kappa/(kappa+dim)
Wi = 0.5/(kappa+dim)

# Prediction 

In [4]:
# Calculating sigma points
x_sigma = np.zeros((x.shape[0], n_sigma))
x_sigma[:,0] = x
S = np.linalg.cholesky(P)
for i in range(1,dim+1):
    x_sigma[:,i] = x + np.sqrt(dim + kappa) * S[:,i-1]
    x_sigma[:,i+dim] = x - np.sqrt(dim + kappa) * S[:,i-1]
    

In [5]:
# state transition function
def f(x,u):
    x_star = np.zeros(np.shape(x))
    for i in range(0,np.shape(x)[1]):
        x_star[0,i] = x[0,i] + x[2,i] + Q[0,0]/2
        x_star[1,i] = x[1,i] + x[3,i] + Q[1,1]/2 
        x_star[2,i] = x[2,i] + Q[0,0]
        x_star[3,i] = x[3,i] + Q[1,1]

    return x_star


In [31]:
# mean covaraince function
def mean_covariance(x_star, P):
    x_pred = Wo * x_star[:,0]
    for i in range(1, n_sigma):
        x_pred = x_pred + Wi * x_star[:,i]
        
    d = (x_star[:,0] - x_pred).reshape(-1,1) 
    P_pred = Wo * (d.dot(d.T))
    
    for i in range(1, n_sigma):
        d = (x_star[:,i] - x_pred).reshape(-1,1)
        P_pred = P_pred + Wi * (d.dot(d.T))
    
    return x_pred, P_pred

In [7]:
# get state transition for every sigma point
x_star = f(x_sigma, u)   

# calculated weighted sum of mean and covariance
x_pred, P_pred = mean_covariance(x_star,P)

In [8]:
P_pred

array([[2.00000000e+00, 4.89093761e-29, 1.00000000e+00, 0.00000000e+00],
       [4.89093761e-29, 2.00000000e+00, 1.45446229e-30, 1.00000000e+00],
       [1.00000000e+00, 1.45446229e-30, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# Correction

In [9]:
# Calculating sigma points
x_pred_sigma = np.zeros((x.shape[0], n_sigma))
x_pred_sigma[:,0] = x_pred
S = np.linalg.cholesky(P_pred)
for i in range(1,dim+1):
    x_pred_sigma[:,i] = x_pred + np.sqrt(dim + kappa) * S[:,i-1]
    x_pred_sigma[:,i+dim] = x_pred - np.sqrt(dim + kappa) * S[:,i-1]

In [26]:
# measurement function
def h(x_pred):
    x_pred_star = np.zeros((np.shape(z)[0],np.shape(x_pred)[1]))
    for i in range(0,np.shape(x_pred)[1]):
        x_pred_star[0, i] = x_pred[0,i] + R[0,0]
        x_pred_star[1, i] = x_pred[1,i] + R[1,1]
        
    return x_pred_star

In [29]:
x_corr_star = h(x_pred_sigma)

In [32]:
x_corr, P_corr = mean_covariance(x_pred_star,R)

In [47]:
def cross_correlation(x_pred, x_pred_sigma, x_corr, x_corr_star):
    
    dx = (x_pred_sigma[:,0]-x_pred).reshape([-1,1])
    dy = (x_corr_star[:,0]-x_corr).reshape([-1,1])
    
    P_pred_x_corr_x = Wo * (dx.dot(dy.T))
    
    for i in range(1, dim):
            dx = (x_pred_sigma[:, i] - x_pred).reshape([-1, 1])
            dy = (x_corr_star[:, i] - x_corr).reshape([-1, 1])
            P_pred_x_corr_x = P_pred_x_corr_x + Wi * (dx.dot(dy.T))
    
    return P_pred_x_corr_x

In [48]:
P_pred_x_corr_x = cross_correlation(x_pred, x_pred_sigma, x_corr, x_corr_star)

In [49]:
P_pred_x_corr_x

array([[1.00000000e+00, 1.45038929e-15],
       [5.80155714e-15, 1.00000000e+00],
       [5.00000000e-01, 1.45038929e-15],
       [2.90077857e-15, 5.00000000e-01]])

In [54]:
K = P_pred_x_corr_x.dot(np.linalg.pinv(P_corr))
        
x = x_pred + (K .dot(z - x_corr))
P = P - (K.dot(P_corr.dot(K.T)))

In [55]:
x

array([74.4 , 29.4 ,  2.25,  1.25])

In [56]:
P

array([[ 5.00000000e-01, -3.62597321e-15, -2.50000000e-01,
        -1.81298661e-15],
       [-3.62597321e-15,  5.00000000e-01, -2.17558393e-15,
        -2.50000000e-01],
       [-2.50000000e-01, -2.17558393e-15,  8.75000000e-01,
        -1.08779196e-15],
       [-1.81298661e-15, -2.50000000e-01, -1.08779196e-15,
         8.75000000e-01]])

In [53]:
K

array([[5.00000000e-01, 7.25194643e-16],
       [2.90077857e-15, 5.00000000e-01],
       [2.50000000e-01, 7.25194643e-16],
       [1.45038929e-15, 2.50000000e-01]])

In [36]:
x_pred_sigma

array([[74.7       , 77.14948974, 74.7       , 74.7       , 74.7       ,
        72.25051026, 74.7       , 74.7       , 74.7       ],
       [29.7       , 29.7       , 32.14948974, 29.7       , 29.7       ,
        29.7       , 27.25051026, 29.7       , 29.7       ],
       [ 2.4       ,  3.62474487,  2.4       ,  3.62474487,  2.4       ,
         1.17525513,  2.4       ,  1.17525513,  2.4       ],
       [ 1.4       ,  1.4       ,  2.62474487,  1.4       ,  2.62474487,
         1.4       ,  0.17525513,  1.4       ,  0.17525513]])

In [37]:
x_corr

array([75.6, 30.6])

In [40]:
x_corr_star

array([[75.6       , 78.04948974, 75.6       , 75.6       , 75.6       ,
        73.15051026, 75.6       , 75.6       , 75.6       ],
       [30.6       , 30.6       , 33.04948974, 30.6       , 30.6       ,
        30.6       , 28.15051026, 30.6       , 30.6       ]])